In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime

In [2]:
spark = SparkSession.builder.master('local[*]').appName('hw_2025').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/13 06:41:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 06:41:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/13 06:41:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Question 1: Install Spark and PySpark
# Execute spark.version

spark.version

#### Answer: 3.2.2

'3.3.2'

In [4]:
# Question 2: Yellow October 2024
# Read the October 2024 Yellow into a Spark Dataframe.
# Repartition the Dataframe to 4 partitions and save it to parquet.
# What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? 
# Select the answer which most closely matches.

!curl -Lf https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -o hw_2025.parquet
df = spark.read.parquet('hw_2025.parquet')
df.repartition(4).write.parquet('hw_2025')
!ls -lash hw_2025/
    
#### Answer: 25 MB

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.3M  100 61.3M    0     0  10.9M      0  0:00:05  0:00:05 --:--:-- 13.3M


[Stage 3:>                                                          (0 + 4) / 4]

total 98M
4.0K drwxr-xr-x 2 ajay ajay 4.0K Jan 13 06:41 .
4.0K drwxrwxr-x 4 ajay ajay 4.0K Jan 13 06:41 ..
4.0K -rw-r--r-- 1 ajay ajay    8 Jan 13 06:41 ._SUCCESS.crc
196K -rw-r--r-- 1 ajay ajay 194K Jan 13 06:41 .part-00000-ea5d5865-b64e-4cb1-b37a-40467315517e-c000.snappy.parquet.crc
196K -rw-r--r-- 1 ajay ajay 194K Jan 13 06:41 .part-00001-ea5d5865-b64e-4cb1-b37a-40467315517e-c000.snappy.parquet.crc
196K -rw-r--r-- 1 ajay ajay 194K Jan 13 06:41 .part-00002-ea5d5865-b64e-4cb1-b37a-40467315517e-c000.snappy.parquet.crc
196K -rw-r--r-- 1 ajay ajay 195K Jan 13 06:41 .part-00003-ea5d5865-b64e-4cb1-b37a-40467315517e-c000.snappy.parquet.crc
   0 -rw-r--r-- 1 ajay ajay    0 Jan 13 06:41 _SUCCESS
 25M -rw-r--r-- 1 ajay ajay  25M Jan 13 06:41 part-00000-ea5d5865-b64e-4cb1-b37a-40467315517e-c000.snappy.parquet
 25M -rw-r--r-- 1 ajay ajay  25M Jan 13 06:41 part-00001-ea5d5865-b64e-4cb1-b37a-40467315517e-c000.snappy.parquet
 25M -rw-r--r-- 1 ajay ajay  25M Jan 13 06:41 part-00002-ea5d5865-b64e-4cb

In [5]:
# Question 3: Count records
# How many taxi trips were there on the 15th of October?
# Consider only trips that started on the 15th of October.

df.filter(F.to_date('tpep_pickup_datetime') == datetime(2024, 10, 15)).count()

#### Answer: 128893

128893

In [6]:
# Question 4: Longest trip
# What is the length of the longest trip in the dataset in hours?

df.groupby(F.to_date('tpep_pickup_datetime').alias('date')).\
    agg(F.max(df.tpep_dropoff_datetime - df.tpep_pickup_datetime).alias('longest')).\
    sort('longest', ascending = False).\
    limit(5).\
    select(F.expr("round(extract(day from longest) * 24 + \
                    extract(hour from longest) + \
                    extract(min from longest) / 60 + \
                    extract(second from longest) / 3600, 2)").\
    alias('longest_in_hours')).show()

#### Answer: 162.62

[Stage 7:==============>                                            (1 + 3) / 4]

+----------------+
|longest_in_hours|
+----------------+
|          162.62|
|          143.32|
|          137.76|
|          114.83|
|            89.9|
+----------------+



In [7]:
# Question 5: User Interface
# Spark’s User Interface which shows the application's dashboard runs on which local port?

df._session

#### Answer: Usually 4040 but mine is running on 4042 since 4040 is occupied.

In [8]:
# Question 6: Least frequent pickup location zone
# Load the zone lookup data into a temp view in Spark:
# wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -o z.csv
zone = spark.read.option('header', 'true').option('inferSchema', 'true').csv('z.csv')
joined = df.join(zone, zone.LocationID == df.PULocationID, how = 'left')
joined.groupby('Zone').count().sort('count').select('zone').head(1)

#### Answer: Governor's Island/Ellis Island/Liberty Island

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  47633      0 --:--:-- --:--:-- --:--:-- 47633


[Row(zone="Governor's Island/Ellis Island/Liberty Island")]

In [9]:
spark.stop()
!rm -rf *.csv *.parquet hw_*